# Import Libraries and Data

In [439]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows", 1000)

import sys
sys.path.append('../../scripts')

from html_parser import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [440]:
sentences = get_fiction_sentences("https://www.gutenberg.org/files/1342/1342-h/1342-h.htm")

In [465]:
sentences[100:110]

['"I hope Mr. Bingley will like it, Lizzy."',
 '"We are not in a way to know what Mr. Bingley likes," said her       mother resentfully, "since we are not to visit."',
 '"But you forget, mamma," said Elizabeth, "that we shall meet him at the       assemblies, and that Mrs. Long promised to introduce him."',
 '"I do not believe Mrs. Long will do any such thing. She has two nieces of       her own. She is a selfish, hypocritical woman, and I have no opinion of       her."',
 '"No more have I," said Mr. Bennet; "and I am glad to find that you do not       depend on her serving you."',
 'Mrs. Bennet deigned not to make any reply, but, unable to contain herself,       began scolding one of her daughters.',
 '"Don\'t keep coughing so, Kitty, for Heaven\'s sake! Have a little       compassion on my nerves. You tear them to pieces."',
 '"Kitty has no discretion in her coughs," said her father; "she times them       ill."',
 '"I do not cough for my own amusement," replied Kitty fretfully. "When

In [442]:
df = pd.DataFrame(sentences, columns=['text'])

index_to_exclude = [i for i in range(1,63)]
df = df.drop(index_to_exclude, axis=0)
df = df.reset_index()
df.shape

(2119, 2)

In [443]:
def get_num_quotes(sent):
    return sent.count('"')
    
def add_tag_to_dict(sent_index, sent, somedict, tag):
    global total_quotes_count
    num_utterance = int(get_num_quotes(sent)/2)
    somedict[sent_index] = (tag, total_quotes_count, num_utterance)
    return somedict
    
def is_quote_start_of_sent(sent):
    # must have capitalized alphabet after quote
    return bool(re.match('^"[A-Z]', sent))

def is_quote_end_of_sent(sent):
    return bool(re.match('.*"$', sent))

def tag_dialogue(sent_index, sent):
    global total_quotes_count, sent_dia_dict, sent_list
    
    num_quotes = get_num_quotes(sent)
    
    if num_quotes == 0:
#         if (total_quotes_count%2) == 0:
        add_tag_to_dict(sent_index, sent, sent_dia_dict, 'narrative')
#         else:
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'dialogue-P')
    
    elif num_quotes == 1:
        total_quotes_count += 1
        add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter')#letter-S
#         if is_quote_start_of_sent(sent):
#             total_quotes_count += 1
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-S')
#         elif is_quote_end_of_sent(sent):
#             total_quotes_count += 1
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-E')
#         else:
#             if (total_quotes_count%2) == 0:
#                 total_quotes_count += 1
#                 add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-S')
#             else: 
#                 total_quotes_count += 1
#                 add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter-E')
                
    elif num_quotes == 2:
#         if (total_quotes_count%2) == 0:
        total_quotes_count += 2
        add_tag_to_dict(sent_index, sent, sent_dia_dict, 'utterance')
#         else:
#             total_quotes_count += 2
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'dialogue-E-S')
        
    elif num_quotes > 2:
#         if (total_quotes_count%2) == 0:
#             total_quotes_count += num_quotes
        if (num_quotes%2) == 0:
            total_quotes_count += num_quotes
            add_tag_to_dict(sent_index, sent, sent_dia_dict, 'utterance')
        else:
            total_quotes_count += num_quotes
            add_tag_to_dict(sent_index, sent, sent_dia_dict, 'letter')
#         else:
#             total_quotes_count += num_quotes
#             add_tag_to_dict(sent_index, sent, sent_dia_dict, 'dialogue-S')

    return sent_dia_dict

In [445]:
doc = df['text'].values

total_quotes_count = 0
sent_dia_dict = dict()
for sent_index, sent in enumerate(doc):
    sent_dia_dict = tag_dialogue(sent_index, sent)

In [446]:
len(df)

2119

In [448]:
df = pd.DataFrame(list(sent_dia_dict.items()), columns=['para_index', 'tag'])

In [449]:
df.shape

(2119, 2)

In [450]:
df['para'] = doc
df['num_quotes_up_to_para'] = df['tag'].apply(lambda x: x[1])
df['num_utterances'] = df['tag'].apply(lambda x: x[2])
df['tag'] = df['tag'].apply(lambda x: x[0])


In [451]:
df['tag'].value_counts()

utterance    1289
narrative    779 
letter       51  
Name: tag, dtype: int64

In [464]:
# df[(df['para'].apply(lambda x: is_quote_start_of_sent(x)==False)) & (df['tag']=='utterance')]

In [453]:
df.loc[100:200]

,para_index,tag,para,num_quotes_up_to_para,num_utterances
100,100,narrative,"The manner in which they spoke of the Meryton assembly was sufficiently characteristic. Bingley had never met with more pleasant people or prettier girls in his life; everybody had been most kind and attentive to him; there had been no formality, no stiffness; he had soon felt acquainted with all the room; and, as to Miss Bennet, he could not conceive an angel more beautiful. Darcy, on the contrary, had seen a collection of people in whom there was little beauty and no fashion, for none of whom he had felt the smallest interest, and from none received either attention or pleasure. Miss Bennet he acknowledged to be pretty, but she smiled too much.",186,0
101,101,narrative,"Mrs. Hurst and her sister allowed it to be so—but still they admired her and liked her, and pronounced her to be a sweet girl, and one whom they would not object to know more of. Miss Bennet was therefore established as a sweet girl, and their brother felt authorized by such commendation to think of her as he chose.",186,0
102,102,narrative,Chapter 5,186,0
103,103,narrative,"Within a short walk of Longbourn lived a family with whom the Bennets were particularly intimate. Sir William Lucas had been formerly in trade in Meryton, where he had made a tolerable fortune, and risen to the honour of knighthood by an address to the king during his mayoralty. The distinction had perhaps been felt too strongly. It had given him a disgust to his business, and to his residence in a small market town; and, in quitting them both, he had removed with his family to a house about a mile from Meryton, denominated from that period Lucas Lodge, where he could think with pleasure of his own importance, and, unshackled by business, occupy himself solely in being civil to all the world. For, though elated by his rank, it did not render him supercilious; on the contrary, he was all attention to everybody. By nature inoffensive, friendly, and obliging, his presentation at St. James's had made him courteous.",186,0
104,104,narrative,"Lady Lucas was a very good kind of woman, not too clever to be a valuable neighbour to Mrs. Bennet. They had several children. The eldest of them, a sensible, intelligent young woman, about twenty-seven, was Elizabeth's intimate friend.",186,0
105,105,narrative,That the Miss Lucases and the Miss Bennets should meet to talk over a ball was absolutely necessary; and the morning after the assembly brought the former to Longbourn to hear and to communicate.,186,0
106,106,utterance,"""You began the evening well, Charlotte,"" said Mrs. Bennet with civil self-command to Miss Lucas. ""You were Mr. Bingley's first choice.""",190,2
107,107,utterance,"""Yes; but he seemed to like his second better.""",192,1
108,108,utterance,"""Oh! you mean Jane, I suppose, because he danced with her twice. To be sure that did seem as if he admired her—indeed I rather believe he did—I heard something about it—but I hardly know what—something about Mr. Robinson.""",194,1
109,109,utterance,"""Perhaps you mean what I overheard between him and Mr. Robinson; did not I mention it to you? Mr. Robinson's asking him how he liked our Meryton assemblies, and whether he did not think there were a great many pretty women in the room, and which he thought the prettiest? and his answering immediately to the last question: 'Oh! the eldest Miss Bennet, beyond a doubt; there cannot be two opinions on that point.'""",196,1


In [1162]:
def find_utterances_raw(para):
    quotes = re.findall('"([^"]*)"', para)
    return ['"' + i + '"' for i in quotes]

def remove_period_from_honorifics(para):
    regex = '(Miss.|Mrs.|Mr.|Ms.)'
    for i in re.findall(regex, para, flags=re.IGNORECASE):
        para = para.replace(i, i[:-1])
    return para

def replace_narrative_w_mark(sent_list):
    new_sent_list = []
    for sent in sent_list:
        if ((get_num_quotes(sent))%2==0) & ((get_num_quotes(sent)!=0)):
            new_sent_list.append(sent)
        else:
            new_sent_list.append('[N]')
    return ' '.join(new_sent_list)

def tokenize_para(para):
    para = remove_period_from_honorifics(para)
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.))'
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.|"[^"]+\?"|"[^"]+\!"))'
    regex = r'(("[^"]*")|([^"]*))'
    results = [i[0] for i in re.findall(regex, para)]
    tag_texts = [i for i in results if i]
    output_string = replace_narrative_w_mark(tag_texts)
    return output_string


            

In [1163]:
teststring0 = '"Come here, child," cried her father as she appeared. "I have sent for you on an affair of importance. I understand that Mr. Collins has made you an offer of marriage. Is it true?" Elizabeth replied that it was. "Very well—and this offer of marriage you have refused?"'
print(teststring0)

"Come here, child," cried her father as she appeared. "I have sent for you on an affair of importance. I understand that Mr. Collins has made you an offer of marriage. Is it true?" Elizabeth replied that it was. "Very well—and this offer of marriage you have refused?"


In [1164]:
tokenize_para(teststring0)

'"Come here, child," [N] "I have sent for you on an affair of importance. I understand that Mr Collins has made you an offer of marriage. Is it true?" [N] "Very well—and this offer of marriage you have refused?"'

In [1169]:
teststring1 = '"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."'
print(teststring1)

"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."


In [1170]:
tokenize_para(teststring1)

'"My dear Mr Bennet," [N] "how can you be so tiresome! You must know that I am thinking of his marrying one of them."'

In [1171]:
find_utterances_raw(teststring1)

['"My dear Mr. Bennet,"',
 '"how can you be so tiresome! You must know that I am thinking of his marrying one of them."']

In [1172]:
teststring2 = '"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"'
print(teststring2)

"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"


In [1173]:
tokenize_para(teststring2)

'"My dear Mr Bennet," [N] "have you heard that Netherfield Park is let at last?"'

In [1174]:
df['raw_utter_list'] = df['para'].apply(lambda x: find_utterances_raw(x))

In [1175]:
df['tokenized_sent'] = df['para'].apply(lambda x: tokenize_para(x))

In [1176]:
df.shape

(2119, 7)

# Cleaning Manual Labeled CSV

In [1359]:
df_labeled = pd.read_csv('../../data/parsed-n-labeled-data/pnp-gutenberg-label-task-pride-and-prejudice-by-jane-austen.csv', index_col=[0])

In [1360]:
df_labeled = df_labeled.reset_index()
df_labeled.columns = ['para_index', 'tag_old','para', 'position', 'speaker', 'type']

In [1361]:
df_labeled['type'] = df_labeled['type'].replace(' c', 'c').replace('c ', 'c').fillna('')

In [1362]:
df_labeled[df_labeled['position'].isnull()]

,para_index,tag_old,para,position,speaker,type
344,344,not,He was silent.,NaN,NaN,
1722,1722,not,Chapter 53,NaN,NaN,


In [1363]:
df_labeled.loc[344, 'position'] = 'n'
df_labeled.loc[1722, 'position'] = 'n'

In [1364]:
df_labeled['position'] = df_labeled['position'].apply(lambda x: x.lower())

In [1365]:
df_labeled.loc[343]

para_index    343                                     
tag_old       dialogue-F                              
para          “How can you contrive to write so even?”
position      1                                       
speaker       NaN                                     
type                                                  
Name: 343, dtype: object

In [1366]:
df_labeled['position'].value_counts()

n    785
1    689
2    507
3    99 
4    29 
5    8  
6    2  
Name: position, dtype: int64

In [1367]:
df_labeled['type'].value_counts()

            1819
c           225 
letter      71  
emphasis    2   
quote       2   
Name: type, dtype: int64

In [1368]:
def recount_convo(type_tag, position_tag):
    global convo_count
    
    if (type_tag == 'c') & (position_tag.lower()!='n'):
        convo_count += 1
        return 'c_{}'.format(convo_count)
    elif (type_tag == '') & (position_tag.lower()!='n'):
        return 'c_{}'.format(convo_count)
    elif (type_tag == 'letter'):
        return 'letter'
    elif (type_tag == 'quote'):
        return 'quote'
    else:
        return 'narrative'



In [1369]:
convo_count = 0
df_labeled['new_type_tag'] = df_labeled.apply(lambda row: recount_convo(str(row['type']), str(row['position'])), axis=1)

In [1370]:
df_labeled.head(10)

,para_index,tag_old,para,position,speaker,type,new_type_tag
0,0,not,By Jane Austen,n,NaN,,narrative
1,1,not,Chapter 1,n,NaN,,narrative
2,2,not,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",n,NaN,,narrative
3,3,not,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",n,NaN,,narrative
4,4,dialogue-F,"“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”",1,mother,c,c_1
5,5,not,Mr. Bennet replied that he had not.,n,NaN,,narrative
6,6,dialogue-F,"“But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.”",1,mother,,c_1
7,7,not,Mr. Bennet made no answer.,n,NaN,,narrative
8,8,dialogue-F,“Do you not want to know who has taken it?” cried his wife impatiently.,1,mother,,c_1
9,9,dialogue-F,"“You want to tell me, and I have no objection to hearing it.”",2,father,,c_1


# Merging

In [1371]:
df_merged = pd.merge(df, df_labeled, on='para_index', how='left')

In [1372]:
df_merged.shape

(2119, 13)

In [1373]:
df_merged.head()

,para_index,tag,para_x,num_quotes_up_to_para,num_utterances,raw_utter_list,tokenized_sent,tag_old,para_y,position,speaker,type,new_type_tag
0,0,narrative,By Jane Austen,0,0,[],[N],not,By Jane Austen,n,NaN,,narrative
1,1,narrative,Chapter 1,0,0,[],[N],not,Chapter 1,n,NaN,,narrative
2,2,narrative,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",0,0,[],[N],not,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",n,NaN,,narrative
3,3,narrative,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",0,0,[],[N],not,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",n,NaN,,narrative
4,4,utterance,"""My dear Mr. Bennet,"" said his lady to him one day, ""have you heard that Netherfield Park is let at last?""",4,2,"[""My dear Mr. Bennet,"", ""have you heard that Netherfield Park is let at last?""]","""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""",dialogue-F,"“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”",1,mother,c,c_1


In [1374]:
columns_i_want = ['para_index', 'para_x', 'num_utterances', 'raw_utter_list', 'tokenized_sent', 'position', 'speaker', 'new_type_tag']
df_merged = df_merged[columns_i_want]

In [1375]:
df_merged = df_merged.fillna('')

In [1376]:
df_merged.columns

Index(['para_index', 'para_x', 'num_utterances', 'raw_utter_list',
       'tokenized_sent', 'position', 'speaker', 'new_type_tag'],
      dtype='object')

In [1377]:
df_merged['chapter_tag'] = df_merged['para_x'].apply(lambda x: x if 'chapter ' in x.lower() else '')

In [1378]:
chapter_dict = dict()
chapter_tag = ''
for i in df_merged.index:
    curr_chapter_tag = df_merged.loc[i]['chapter_tag']
    if curr_chapter_tag == '':
        if chapter_tag != '':
            chapter_dict[i] = chapter_tag
        else:
            chapter_tag = curr_chapter_tag
            chapter_dict[i] = chapter_tag
    else: 
        if chapter_tag == curr_chapter_tag:
            chapter_dict[i] = chapter_tag
        else:
            chapter_tag = curr_chapter_tag
            chapter_dict[i] = chapter_tag



In [1379]:
df_merged['new_chapter_tag'] = list(x[1] for x in chapter_dict.items())

In [1380]:
def is_utterance(sent):
    return get_num_quotes(sent)%2==0


def tag_tokens(sent_list):
    new_list = []
    for sent in sent_list:
        if is_utterance(sent):
            new_list.append((sent, ))

In [1382]:
new_label_dict = dict()
count = 0 
for j in df_merged.index:
    tuple_i_want = (j,) \
                + tuple(list(df_merged.loc[j][['position', 'speaker', 'new_type_tag', 'new_chapter_tag']].values)) \
                + (tokenize_para(df_merged.loc[j]['para_x']),)
    new_label_dict[count] = tuple_i_want
    count += 1     

In [1383]:
df_final = pd.DataFrame(list(new_label_dict.items()), columns=['sent_index', 'tag'])

In [1385]:
df_final['para_index'] = df_final['tag'].apply(lambda x: x[0])
df_final['position_tag'] = df_final['tag'].apply(lambda x: x[1])
df_final['speaker_tag'] = df_final['tag'].apply(lambda x: x[2])
df_final['type_tag'] = df_final['tag'].apply(lambda x: x[3])
df_final['chapter_tag'] = df_final['tag'].apply(lambda x: x[4])
df_final['tokenized_sent'] = df_final['tag'].apply(lambda x: x[5])


In [1386]:
df_final.head()

,sent_index,tag,para_index,position_tag,speaker_tag,type_tag,tokenized_sent,chapter_tag
0,0,"(0, n, , narrative, , [N])",0,n,,narrative,[N],
1,1,"(1, n, , narrative, Chapter 1, [N])",1,n,,narrative,[N],Chapter 1
2,2,"(2, n, , narrative, Chapter 1, [N])",2,n,,narrative,[N],Chapter 1
3,3,"(3, n, , narrative, Chapter 1, [N])",3,n,,narrative,[N],Chapter 1
4,4,"(4, 1, mother, c_1, Chapter 1, ""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?"")",4,1,mother,c_1,"""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""",Chapter 1


In [1387]:
index_to_exclude = df_final[df_final['tokenized_sent'].apply(lambda x: x=='[N]')].index

In [1388]:
df_final = df_final.drop(index_to_exclude, axis=0)

In [1389]:
df_final = df_final.reset_index()
del df_final['index']

In [1391]:
df_final.shape

(1290, 8)

In [1392]:
label_dict = dict()

conver_tag = ''
position_tag = 'n'

for i in df_final.index:
    curr_position_tag = df_final.loc[i]['position_tag']
    curr_conver_tag = df_final.loc[i]['type_tag']
    if (curr_conver_tag == conver_tag) & (curr_position_tag!='n'):
        if curr_position_tag == position_tag:
            label_dict[i] = 'part'
            position_tag = curr_position_tag
            conver_tag = curr_conver_tag
        else:
            label_dict[i] = 'response'
            position_tag = curr_position_tag
            conver_tag = curr_conver_tag
    else: 
        label_dict[i] = 'not'
        position_tag = curr_position_tag
        conver_tag = curr_conver_tag    

In [1394]:
df_final['label'] = list(x[1] for x in label_dict.items())

In [1395]:
df_final.head()

,sent_index,tag,para_index,position_tag,speaker_tag,type_tag,tokenized_sent,chapter_tag,label
0,4,"(4, 1, mother, c_1, Chapter 1, ""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?"")",4,1,mother,c_1,"""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""",Chapter 1,not
1,6,"(6, 1, mother, c_1, Chapter 1, ""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it."")",6,1,mother,c_1,"""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""",Chapter 1,part
2,8,"(8, 1, mother, c_1, Chapter 1, ""Do you not want to know who has taken it?"" [N])",8,1,mother,c_1,"""Do you not want to know who has taken it?"" [N]",Chapter 1,part
3,9,"(9, 2, father, c_1, Chapter 1, ""You want to tell me, and I have no objection to hearing it."")",9,2,father,c_1,"""You want to tell me, and I have no objection to hearing it.""",Chapter 1,response
4,11,"(11, 1, , c_1, Chapter 1, ""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week."")",11,1,,c_1,"""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.""",Chapter 1,response


In [1396]:
df_final.shape

(1290, 9)